## Network Failures - Servicer

- **Goal**: to understand the impact of parts of the servicer network 'going dark' on the ability of the network to reassign relay requests (work)
- **System Goals Targeted**:
  - Service Reliability
  - Accessibility
- **Design**: allow for groups of servicers to be rendered unavailable by 1) 'shutting down' by attribute, such as GeoZone, service type, etc., or 2) 'unlinking' servicers from services
- **Testing mechanism**:
  - All servicers of with a matching attribute do not perform the work they are allocated
  - Allow for a random selection of servicers to lose their connection to a given service, for a certain duration (perhaps infinite)
  - The overall loss in QoS is measured with respect to the 'self healing' properties of the network to reallocate work, by assessing 
    - Different servicer-service topologies (e.g. network densities)
- **KPIs**:
  - Servicer slashing cost (KPI-8)
  - Servicer jailing opportunity cost (KPI-11)
  - Network load balancing (KPI-C)
  - Application relay cost per byte (KPI-6)

## Protocol Parameters to be swept

1. `SlashFractionDowntime` (`SFD`)
2. `MaxChainsPerServicer` (`MCPS`)
3. `DowntimeJailDuration` (`DJD`)
4. `MaxJailedBlocks` (`MJB`)

## Exogenous Parameters to be swept
1. Servicer-to-service network density parameter: $\delta \in [0,1]$
2. Attribute to condition upon for shutdown: $A$, with value taken from the set of servicer attributes (e.g. GeoZone, service type etc.)
3. Duration of a service (chain) outage: $D$ (in days)

## Simulation Initialization

1. Initialize a set $N_s$ of servicers
2. Initialize a set $N_c$ of services (chains)
3. Initialize a set $N_a$ of applications
4. Initialize a set $N_g$ of gateways
5. Initialize the network of applications - gateways - services - servicers
    - Initialize to a particular servicer-to-service density, given by the sweep value of the $\delta$ density parameter. This requires explicitly adding services (chains) to servicers until the density $\delta$ is met. This in turn means adding services to servicers until the number of such connections, $N$, is equal to
    $$N := \lfloor \delta \times |N_s| \times | N_c | \rfloor,$$
    where $\lfloor \cdot \rfloor$ is the 'floor' operator (lowest nearest integer), and $| \cdot |$ is the number of elements of the set.
7. Initialize the number of timesteps $T$ even


## Simulation routine A - servicer shutdown by attribute
Simulation runs with default behavior for $T/2$ timesteps, to 'burn in' QoS when all servicers are operational

In timestep $T/2 + 1$:
1. For each servicer matching attribute $A$, remove all of their connections to services--these servicers are no longer active for the remainder of the simulation

**Simulation then runs for the remaining $T/2$ timesteps, with the network operating only using servicers that do not match the selected attribute. Servicers which are no longer active are assessed their slashing and/or jailing penalties, as appropriate.**

## Simulation routine B - random service shutdown
Simulation runs with default behavior for $T/2$ timesteps, to 'burn in' QoS when all servicers are operational

In timestep $T/2 + 1$:
1. Randomly select one active service (chain) $C$, and remove all of its connections to servicers--these servicers are no longer able to access the affected service $C$ for $D$ timesteps

**Simulation then runs for $T/2 + D$ timesteps, with the network operating without access to service $C$.**

In timestep $T/2 + D + 1$:
1. For inactive service $C$, restore all of its pre-existing connections to servicers--these servicers are now able to once again access service $C$

**Simulation then runs for remaining $T - T/2 - D$ timesteps, with the network operating in the same topological state as in the first $T/2$ timesteps.**

Parameter Sweep Values (7 parameters in total)

|  Name | Sweep Values | Units |
| --- | ---| ---|
| `SFD` | (1e-10, 1e-6, 1e-1) | seconds |
| `MCPS` | (1, 12, 20) | number |
| `DJD` | (60, 3600, 28800) | seconds |
| `MJB`  | (200,400,800) | days |
| $\delta$  | (0.1, 0.5, 1.) | % |
| $A$  | ('Geozone', ?, ?) | string |
| $D$  | (1, 7, $T$) | days |

**Total number of parameter constellations**: 
  - Routine A: $3^{7} = 2,187$
  - Routine B: $3^{6} = 729$ (excludes attribute $A$ sweep)

**Total number of Monte Carlo runs per constellation**: ? 30

**Total number of experiments**: 84,480